## Build Goal Refinement Graph

This notebook builds a goal refinement graph from the goals sampled from agglomerative clustering.

In [1]:
from openai import OpenAI

client = OpenAI()

def prompt_model(prompt):
    response = client.chat.completions.create(
      model="gpt-4o-2024-08-06",
      messages=[
        {
          "role": "system",
          "content": "You are a business analyst building a goal model of stakeholder goals for a software application. Goal models are directed, acyclic graphs in which edges trace from high-level goals to low-level goals through refinement relationships. High-level goals describe what states stakeholders want to achieve, maintain or avoid in the system. Low-level goals describe *how* the system will satsify high-level goals, tend to be more specific and describe how the system will operate. High-level goals describe *why* the system aims to satisfy low-level goals, tend to be more generic and describe what the stakeholder aims to accomplish independent of a specific software application. "
        },
        {
          "role": "user",
          "content": prompt
        }
      ]
    )
    return response.choices[0].message.content

In [2]:
import json

data_path = 'data1'

sampled = json.load(open('%s/summarized.json' %  data_path, 'r'))

In [3]:
for i, g in enumerate(sampled['5']):
    print('%s: %s' % (i, g))

0: Distract from the stress of the real world.
1: Create a sense of ongoing activity in the background.
2: Enable users to access their playlists easily.
3: Allow users to create and manage playlists.
4: Support the sharing of playlists with others.
5: Enable users to like playlists.
6: Provide personalized playlists for users.
7: Allow users to open the application.
8: Enable access to the playlist section.
9: Allow users to select and play a specific playlist.
10: Allow users to shuffle songs within a playlist.
11: Enable shuffle mode for the playlist.
12: Enable users to repeat a playlist once it has ended.
13: Enable repeat mode once the entire playlist is over.
14: Search for songs based on emotions or feelings using keywords.
15: Search for public playlists that align with specific emotions or feelings.
16: Explore playlists and songs to find inspiration for dancing.
17: Discover new music.
18: Enhance social interaction through music.
19: Allow users to listen to a particular so

In [4]:
# The program describes a goal graph, in which each goal describes a state in the system to be achieved, maintained or avoided. The functions define refinement relationships between two goals X and Y. When goal X is refined by goal Y, then goal Y must be satisfied in order to satisfy the goal X. Refinement relationships are asymmetric. When a goal X is refined by a goal Y, then the goal Y explains how the goal X is satisfied. The goal Y could be one of several steps that must be performed to satisfy goal X, or it could be one of many cases that must be handled to satisfy goal X. Refinement relationships are transitive, thus if a goal Z satisfies goal Y and goal Y satisfies goal X, then it is also true that Z satisfies X. 

prompt_base = """Read the following Python code that describes an initial goal model and complete the code using the following function calls. Before completing the code, perform the following steps: 1) read all of the goals and describe what the software application is and does; 2) for each goal, describe what the goal means in the context of the application description, including what is and is not intended by the goal description; and 3) complete the code by considering which goals are refined by other goals. Include your justification for each function call in comments. When responding, include the python code between the start ```python and end ``` tags.
1. X.is_refined_by.append(Y) - when the goal X is satisfied by the goal Y and has the refinement goal Y
2. Y.is_refinement_of(X) - when the goal Y satisfies the goal X and is a refinement of goal X
%s
"""

prompt_text = """Read the following Python code that describes an initial goal model and complete the code using the following function calls. Before completing the code, perform the following steps: 1) read all of the goals and describe what the software application is and does; 2) for each goal, describe what the goal means in the context of the application description, including what is and is not intended by the goal description; and 3) complete the code by adding your implied goals and then considering which goals are refined by other goals. Include your justification for each function call in comments. When responding, include the python code between the start ```python and end ``` tags.
1. X.is_why_we_satisfy_the_goal.append(Y) - when the goal X is satisfied by the goal Y and has the refinement goal Y
2. Y.is_how_we_satisfy_the_goal(X) - when the goal Y satisfies the goal X and is a refinement of goal X
%s
"""

prompt_w_implied = """Read the following Python code that describes an initial goal model and complete the code using the following function calls. Before completing the code, perform the following steps: 1) read all of the goals and describe what the software application is and does; 2) for each goal, describe what the goal means in the context of the application description, including what is and is not intended by the goal description; 3) identify up to two or three *implied* goals that were not included in the original goal list and that add missing context to the original goals; 4) extend the code by adding your implied goals; and 5) complete the code by reviewing all of the goals and deciding which goals are refined by other goals. Implied goals include high-level goals that group related refinements together, and explain what actions the low-level goals seek to achieve. For each implied goal that you create, add the goal to the list of implied goals. Include your justification for each function call in comments. Do not write code to print the goals. When responding, include the python code between the start ```python and end ``` tags.
1. X.is_refined_by.append(Y) - when the goal X is satisfied by the goal Y and has the refinement goal Y
2. Y.is_refinement_of(X) - when the goal Y satisfies the goal X and is a refinement of goal X
3. implied_goals.append(X) - to collect the implied goals that you created
%s
"""

def sample_code(prefix_code, verbose=False):
    p = prompt_w_implied % '\n'.join(prefix_code)
    if verbose:
        print(p)
    r = prompt_model(p)
    if verbose:
        print(r)

    return p, r

In [5]:
# create the class definition code lines
def create_class_def():
    class_def_base = [
        'class Goal:', 
        '    def __init__(self, text):', 
        '        self.text = text', 
        '        self.is_refined_by = []',
        '    def is_refinement_of(self, goal):',
        '        goal.is_refined_by.append(self)'
    ]
    class_def_text = [
        'class Goal:', 
        '    def __init__(self, text):', 
        '        self.text = text', 
        '        self.is_why_we_satisfy_the_goal = []',
        '    def is_how_we_satisfy_the_goal(self, goal):',
        '        goal.is_why_we_satisfy_the_goal.append(self)'
    ]
    class_def_implied = [
        'class Goal:', 
        '    def __init__(self, text):', 
        '        self.text = text', 
        '        self.is_refined_by = []',
        '    def is_refinement_of(self, goal):',
        '        goal.is_refined_by.append(self)',
        'implied_goals = []'
    ]
    return class_def_implied

# create the goal instances
def create_prefix_code(g_list, var_map):
    prefix_code = []
    for g in g_list:
        prefix_code.append('%s = Goal("%s")' % (var_map[g], g))
    return prefix_code

# extract code from model response
def match_code(text):
    match = []
    i = text.find('```python\n')
    while i >= 0:
        j = text.find('```', i + 1)
        match.extend(text[i+10:j].split('\n'))
        i = text.find('```python\n', j + 3)
    return match

In [6]:
def collect_responses(goals, verbose=False):
    # create initial class definition
    code = create_class_def()

    # create variable name map
    var_map = {g: 'g%s' % i for i, g in enumerate(goals)}

    # create goal definitions
    prefix_code = create_prefix_code(goals, var_map)

    response = {'pass': [], 'fail': []}
    passed = False
    failed = 0
    while not passed and failed < 3:
        try:
            p, r = sample_code(code + prefix_code)
            if verbose:
                print(p)
                print(r)
                print()

            # parse suffix code from response
            suffix_code = match_code(r)
            if len(suffix_code) == 0:
                raise Exception('Response did not a contain python match.')
                
            # include prefix, which may be omitted in response
            program = code + prefix_code + suffix_code
        
            # create symbol table from generated code
            local = {}
            exec('\n'.join(program), {}, local)
            passed = True
                
            # save responses for post-processing
            response['pass'] = [p, r, program]

            # append code for follow-on prompt
            code += prefix_code + suffix_code

            print(' .', end='')
        
        except Exception as e:
            if verbose:
                print('Error: %s' % e)
                print(p)
                print(r)
                print()
            response['fail'].append([p, r, str(e)])
            failed += 1
            print(' !', end='')
            
    if not passed:
        print(' F', end='')
                
    return response

In [7]:
import os

results = {}
for i, goals in sampled.items():
    if os.path.exists('%s/cache/refinements-%s-implied.json' % (data_path, i)):
        r = json.load(open('%s/cache/refinements-%s-implied.json' % (data_path, i), 'r'))
        if len(r) > len(results):
            results = r
print(len(results))
for i, r in results.items():
    print('%s: %i' % (i, len(r)))

33
32: 10
35: 10
34: 10
33: 10
20: 10
18: 10
27: 10
9: 10
11: 10
7: 10
29: 10
16: 10
6: 10
28: 10
17: 10
10: 10
19: 10
26: 10
8: 10
21: 10
36: 10
31: 10
30: 10
24: 10
23: 10
4: 10
15: 10
3: 10
12: 10
13: 10
5: 10
14: 10
22: 10


In [8]:
for i, goals in sampled.items():
    if i in results:
        continue
        
    results[i] = []
    print('Building graph goals %s (%i goals)' % (i, len(goals)), end='')
    for j in range(10):
        result = collect_responses(goals, verbose=False)
        results[i].append(result)

    json.dump(results, open('%s/cache/refinements-%s-implied.json' % (data_path, i), 'w+')) 
    print(' .done!')

Building graph goals 25 (30 goals) . . . . . . . . . . .done!


In [9]:
print(len(results))

34


In [10]:
import json
json.dump(results, open('%s/models.json' % data_path, 'w+'))

In [11]:
print(results['4'][2]['pass'][1])

Upon reviewing the goals provided in the initial goal model, the software application appears to be a comprehensive digital banking application. It facilitates a wide range of financial services and transactions, enabling users to manage their accounts, make payments, track finances, and access customer support efficiently. 

### Understanding and Analyzing the Goals:

1. **g0**: Focuses on making cost-sharing among friends easier using digital payments. It suggests peer-to-peer payment features but doesn't imply retail transactions.

2. **g1**: Extends the digital payment method to retail outlets, indicating support for both peer-to-peer and business transaction payment locations.

3. **g2**: Emphasizes instant money transfers using phone numbers, meaning peer-to-peer transactions are prioritized for speed and ease of use.

4. **g3**: Allows direct communication with representatives through the website, suggesting a robust support system.

5. **g4**: Broadly covers fund transfers betw